In [1]:
# Importing the nltk library
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Importing all the necessary modules
import numpy
import tflearn
import tensorflow
import random
import json
import pickle

2022-02-07 18:03:16.398648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-07 18:03:16.398692: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term
Scipy not supported!


In [3]:
# Train the model based on the data
def train(data, chatbot_type):
    
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])
            
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    
    labels = sorted(labels)
    
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)
    
    tensorflow.compat.v1.reset_default_graph()

    net = tflearn.input_data(shape=[None, len(training[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
    net = tflearn.regression(net)

    model = tflearn.DNN(net)
    
    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model_name = "model_" + chatbot_type + ".tflearn"
    model.save(model_name)

In [4]:
# Load the earthquake intents
with open("earthquake_intents.json") as file:
    data = json.load(file)

# Train the model
train(data, "earthquake")

Training Step: 4999  | time: 0.009s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.9998 -- iter: 32/35
Training Step: 5000  | time: 0.012s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.9998 -- iter: 35/35
--
INFO:tensorflow:/home/studio-lab-user/chatbots-notebook/model_earthquake.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [5]:
# Load the firstaid intents
with open("firstaid_intents.json") as file:
    data = json.load(file)

# Train the model
train(data, "firstaid")

Training Step: 23999  | time: 0.065s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 184/188
Training Step: 24000  | time: 0.068s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 188/188
--
INFO:tensorflow:/home/studio-lab-user/chatbots-notebook/model_firstaid.tflearn is not in all_model_checkpoint_paths. Manually adding it.
